<a href="https://colab.research.google.com/github/Shopping-Yuan/ML2021HW/blob/Shopping_vscode_branch/HW2/ML2021HW2_modified.ipynb" target="_parent">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download the dataset
# You may choose where to download the data.

# Google Drive
#!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

# Dropbox
# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip

# MEGA
# !sudo apt install megatools
# !megadl "https://mega.nz/#!zt1TTIhK!ZuMbg5ZjGWzWX1I6nEUbfjMZgCmAgeqJlwDkqdIryfg"

# Unzip the dataset.
# This may take some time.
#!unzip -q food-11.zip

In [29]:
#import pytorch
import torch

# torch.backends.cudnn: set CNN algorithmtorch.backends.cudnn
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# get the current available device ('cpu' or 'cuda')
def get_device():
#    return 'cpu'
    return 'cuda' if torch.cuda.is_available() else 'cpu'
device = get_device()
print(torch.cuda.is_available())
#set random variable
import numpy as np
myseed = 1
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

True


In [3]:
# import torch
# import torch.nn as nn
# m = nn.Conv2d(1, 1, 2 , stride=2, padding=(9,9),bias = False)
# test = torch.tensor([[[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]])
# print(test)
# print(m(test))

In [4]:
# from PIL import Image
# with Image.open("C:/Users/user/Pictures\Screenshots/資格證.jpg") as im:
#     im.show()

In [5]:
# import torch
# x = torch.FloatTensor([[2,7,1],[3,4,9],[5,6,8]])
# y = x[x> 2]
# print(x,y)
# print(x.argmax(dim = 0))
# print(x.argmax(dim = 1))
# print(x.argmax(dim = -1))

In [71]:
import torchvision.transforms as transforms
from torchvision.datasets import DatasetFolder
from PIL import Image
def get_tfm(mode):
    if mode in ["val","test"]:
      tfm = transforms.Compose([
      transforms.Resize(data_info["size"]),
      transforms.ToTensor(),
      ])
    elif mode in ["train_unlabeled","train"]:
       tfm = transforms.Compose([
      transforms.Resize(data_info["size"]),
      transforms.ToTensor(),
      ])
    return tfm
def food_f(mode):
    dataset = DatasetFolder(data_info[mode]["path"], loader=lambda x: Image.open(x), extensions="jpg", transform=get_tfm(mode))
    print('Size of {} data: {}'.format(mode,len(list(dataset))))
    return dataset

#create a dict of functions and path w.r.t. different mode
data_info = {
    "train_origin":{"path":"./food-11/training/labeled"},
    "train":{"path":"./food-11/training/unlabeled"},
    "val":{"path":"./food-11/validation"},
    "test":{"path":"./food-11/testing"},
    "size" :(128,128)
}

In [31]:

tfm = transforms.Compose([
      transforms.Resize(data_info["size"]),
      transforms.ToTensor(),
      ])
print(type(tfm))

<class 'torchvision.transforms.transforms.Compose'>


In [32]:
x = 500# x = int(max(prep))
print(list(food_f("val"))[x][1])

Size of val data: 660
8


In [53]:
h_paras = {
    # maximum number of epochs
    'n_epochs': 2,
    # mini-batch size for dataloader
    'batch_size': 1024,
    # optimization algorithm (optimizer in torch.optim)
    'optimizer': 'Adam',
    # hyper-parameters for the optimizer (depends on which optimizer you are using)
    'optim_hparas': {
        # learning rate of Adam
        'lr': 0.0003,
        "weight_decay" : 1e-5
    },
    # your model will be saved here
    'save_path': './model.pth'
}

In [69]:
from torch.utils.data import DataLoader

# Construct data loaders.
train_set = DataLoader(food_f("train"), batch_size=h_paras["batch_size"], shuffle=True, num_workers=0, pin_memory=True)
valid_set = DataLoader(food_f("val"), batch_size=h_paras["batch_size"], shuffle=True, num_workers=0, pin_memory=True)
test_set = DataLoader(food_f("test"), batch_size=h_paras["batch_size"], shuffle=False)

Size of train data: 6786
Size of val data: 660
Size of test data: 3347


In [35]:
for i , t in enumerate(valid_set):
    if i ==0:
        test_t = t[0]
#        print(type(t[0]),t[0].size())
        print(test_t.size())
        x = test_t[list(range(16))]
        print(x.size())

torch.Size([128, 3, 32, 32])
torch.Size([16, 3, 32, 32])


In [36]:
from torch import nn
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 32, 32]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),
        )
        self.flat_layer = nn.Flatten()
        self.fc_layers = nn.Sequential(
            nn.Linear(int(256 * data_info["size"][0]/8 * data_info["size"][1]/8), 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )
        self.criterion = nn.CrossEntropyLoss()
    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = self.flat_layer(x)

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x
    def cal_loss(self, pred, target):
        return self.criterion(pred, target)

In [37]:
#%conda install ipywidgets

In [38]:
from tqdm.auto import tqdm
from torch.utils.data import ConcatDataset
def get_pseudo_labels(model, threshold=0.65):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.

    # Construct a data loader.
    train_unlabeled_set = \
    DataLoader(food_f("train_unlabeled"), batch_size=h_paras["batch_size"], shuffle=True, num_workers=8, pin_memory=True)
    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)

    # Iterate over the dataset by batches.
    datas = []
    labels = []
    for batch in tqdm(train_unlabeled_set):
        img = batch[0]
        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            img_d = img.to(device)
            logits = model(img_d)

        # Obtain the probability distributions by applying softmax on logits.
        predicts = softmax(logits)
        for  row_index , probs in enumerate(predicts):
            max_prob , max_column_index = probs.max(dim = -1)
            if max_prob > threshold:
                datas.append(img[row_index].tolist())
                labels += (max_column_index.tolist())
    new_train_set = torch.FloatTensor(datas)
    new_train_label = torch.LongTensor(labels)
    model.train()
    return new_train_set,new_train_label

In [62]:
def train(train_set,model_d,optimizer,device):
    # set model to training mode
    model_d.train()
    total_correct_number = 0
    train_loss_list = []
    # iterate through the dataloader
    for data , label in train_set:
      # move data to device (cpu/cuda)
      data_d , label_d = data.to(device), label.to(device)
      # forward pass (compute output tensor)
      pred = (model_d(data_d))
      # get the index of the class with the highest probability
      max_prob_values, max_prob_indexs = torch.max(pred, dim = 1)
      correct_number = (max_prob_indexs.cpu() == label_d.cpu()).sum().item()
#      print(correct_number)
      total_correct_number += correct_number
      # compute loss
      loss = model_d.cal_loss(pred , label_d)
      # compute gradient (backpropagation)
      loss.backward()
      # Clip the gradient norms for stable training.
      nn.utils.clip_grad_norm_(model_d.parameters(), max_norm=10)
      # update model with optimizer
      optimizer.step()
      # set optimizer gradient to zero
      optimizer.zero_grad()
      train_loss_list.append(loss.detach().cpu().item())
    acc = total_correct_number/len(train_set.dataset) 
#    print(acc , train_loss_list)   
    return acc , train_loss_list

In [55]:
def val(val_set,model_d,device):
    # set model to evalutation mode
    model_d.eval()
    total_correct_number = 0
    total_loss = 0
    # iterate through the dataloader
    for data , label in val_set:
    # move data to device (cpu/cuda)
      data_d, label_d = data.to(device), label.to(device)
      # disable gradient calculation
      with torch.no_grad():
        # forward pass (compute output)
        pred = model_d(data_d)
        # get the index of the class with the highest probability
        max_prob_values, max_prob_indexs = torch.max(pred, dim = 1)
        total_correct_number += (max_prob_indexs.cpu() == label_d.cpu()).sum().item()
        # compute loss
        mse_loss = model_d.cal_loss(pred, label_d)
      # accumulate loss
      batch_size = len(data_d)
      total_loss += mse_loss.detach().cpu().item() * batch_size

    # compute averaged loss
    totol_size = len(val_set.dataset)
    acc = total_correct_number/totol_size
    avg_loss =  total_loss/totol_size
    return acc, avg_loss

In [66]:
def train_val_process(training_set, validation_set, model, h_paras, device):
    # Initialize a model, and put it on the device specified.
    model_d = model.to(device)
    model_d.device = device
    optimizer = torch.optim.Adam(model_d.parameters(), **h_paras["optim_hparas"])

    # The number of training epochs.
    n_epochs = h_paras["n_epochs"]

    # record training accuracy
    acc_record = {'train': [], "val": []}
    # record training loss
    loss_record = {'train': [], "val": []}

    # Whether to do semi-supervised learning.
    do_semi = False
    train_set_origin = food_f("train")
    for epoch in range(n_epochs):
        # ---------- Training ----------
        train_acc , train_loss_list = train(training_set,model_d,optimizer,device)
        # save accuracy to acc_record['train']
        acc_record['train'].append(train_acc)
      # save loss to loss_record['train']
        loss_record['train'].append(train_loss_list)

        if do_semi:
            # Obtain pseudo-labels for unlabeled data using trained model.
            pseudo_set = get_pseudo_labels(model_d)
            concat_dataset = ConcatDataset([train_set_origin, pseudo_set])
            train_set_origin = pseudo_set
            training_set = DataLoader(concat_dataset, batch_size=128, shuffle=True, num_workers=8, pin_memory=True)

        # ---------- Validation ----------
        val_acc , val_loss = val(validation_set,model_d,device)
        # Print the information.
        acc_record["val"].append(val_acc)
        # save loss to loss_record["val"]
        loss_record["val"].append(val_loss)
        
        print(' model epoch = {:4d}, train_loss = {:.4f} , val_loss = {:.4f})'\
        .format(epoch , train_loss_list[-1] , val_loss))
        print('train set accuracy = {:.3f}'.format(train_acc))
        torch.save(model.state_dict(), h_paras["save_path"])
        print('saving model with acc {:.3f}'.format(val_acc))
    return acc_record , loss_record


In [70]:
# Construct model
model = Classifier()
model_acc_record, model_loss_record = train_val_process(train_set, valid_set, model, h_paras, device)

Size of train data: 6786


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacty of 2.00 GiB of which 0 bytes is free. Of the allocated memory 4.99 GiB is allocated by PyTorch, and 312.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_set):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        pred = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(pred.argmax(dim=-1).cpu().numpy().tolist())

In [ ]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")